# Data access in Batch Endpoints

In [3]:
from azure.ai.ml import MLClient, Input
from azure.ai.ml.entities import (
    BatchEndpoint,
    BatchDeployment,
    Model,
    AmlCompute,
    Data,
    BatchRetrySettings,
    CodeConfiguration,
    Environment,
    ManagedIdentityConfiguration,
)
from azure.ai.ml.constants import (
    AssetTypes,
    BatchDeploymentOutputAction,
    ManagedServiceIdentityType,
)
from azure.identity import DefaultAzureCredential

In [4]:
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace = "<AML_WORKSPACE_NAME>"

ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

In [6]:
compute_name = "cpu-cluster"
compute_cluster = ml_client.compute.get(name=compute_name)

In [5]:
endpoint_name = "heart-classifier-batch"
endpoint = ml_client.batch_endpoints.get(endpoint_name)

In [8]:
deployment_name = "classifier-xgboost"
deployment = ml_client.batch_deployments.get(
    endpoint_name=endpoint_name, name=deployment_name
)

In [14]:
default_ds = ml_client.datastores.get_default()
default_ds.id

'/subscriptions/18522758-626e-4d88-92ac-dc9c7a5c26d4/resourceGroups/Analytics.Aml.Experiments.Workspaces/providers/Microsoft.MachineLearningServices/workspaces/aa-ml-aml-workspace/datastores/workspaceblobstore'

In [25]:
job = ml_client.batch_endpoints.invoke(
    endpoint_name=endpoint_name,
    inputs={
        "mnist": Input(
            path="https://azuremlexampledata.blob.core.windows.net/data/heart-disease-uci/data",
            type=AssetTypes.URI_FOLDER,
        )
    },
    output_path={
        "mnist": Input(
            path="https://azuremlexampledata.blob.core.windows.net/data/heart-disease-uci/data",
            type=AssetTypes.URI_FOLDER,
        )
    },
)

In [26]:
job = ml_client.batch_endpoints.invoke(
    endpoint_name=endpoint_name,
    inputs={
        "mnist": Input(
            path="https://azuremlexampledata.blob.core.windows.net/data/heart-disease-uci/data",
            type=AssetTypes.URI_FOLDER,
        )
    },
    output_file_name="predictions.csv",
)

In [13]:
import inspect

inspect.signature(ml_client.batch_endpoints.invoke)

<Signature (endpoint_name: str, *, deployment_name: str = None, inputs: Dict[str, azure.ai.ml.entities._inputs_outputs.input.Input] = None, **kwargs) -> azure.ai.ml.entities._deployment.batch_job.BatchJob>

In [22]:
compute_cluster.identity.type = ManagedServiceIdentityType.USER_ASSIGNED
compute_cluster.identity.user_assigned_identities = [
    ManagedIdentityConfiguration(
        resource_id="/subscriptions/18522758-626e-4d88-92ac-dc9c7a5c26d4/resourcegroups/analytics.aml.experiments.workspaces/providers/Microsoft.ManagedIdentity/userAssignedIdentities/azureml-cpu-cluster-idn"
    )
]

In [23]:
ml_client.compute.begin_create_or_update(compute_cluster)

AttributeError: 'NoneType' object has no attribute 'lower'

In [25]:
endpoint_name = "heart-classifier-batch"

In [26]:
endpoint = ml_client.batch_endpoints.get(endpoint_name)

## Using a datastore path

In [8]:
default_ds = ml_client.datastores.get_default()

In [10]:
default_ds.id

'/subscriptions/18522758-626e-4d88-92ac-dc9c7a5c26d4/resourceGroups/Analytics.Aml.Experiments.Workspaces/providers/Microsoft.MachineLearningServices/workspaces/aa-ml-aml-workspace/datastores/workspaceblobstore'

### Credential-lesss store

In [30]:
input = Input(
    type=AssetTypes.URI_FOLDER,
    path="azureml://datastores/amldatalake_trusted_cl/paths/datasets/heart-disease-uci-unlabeled/",
)

In [31]:
job = ml_client.batch_endpoints.invoke(endpoint_name=endpoint.name, input=input)

In [32]:
ml_client.jobs.get(job.name)

Experiment,Name,Type,Status,Details Page
heart-classifier-batch,5753e564-4ce1-4d07-a37c-b670fca09ea9,pipeline,Running,Link to Azure Machine Learning studio
